# PANDA: baseline

This notebook got inspirations from
- [PANDA 16X128X128 tiles from Iafoss](https://www.kaggle.com/iafoss/panda-16x128x128-tiles) 
- [PANDA keras baseline from Noble](https://www.kaggle.com/nobletp/panda-keras-baseline)

Thank you for sharing!

This model uses quadratic weighted kappa (QWK) as loss and metric, as implemented in TensorFlow Addons.

- Version 1  Only the top fully connected layer is trained. Lower layers from EfficientNetB3 are fixed at ImageNet weights. QWKs for train and validation set are both 0.54.
- Version 2  Top layers from 'block7b_add' and above from EfficientNetB3 base model are also trained.
    - ValueError: axes don't match array
- Version 4  Load saved weights from Version 1 using the same model as Version 1. Then set more layers trainable.

Set some parameters.

In [ ]:
DATA_PATH     = '/kaggle/input/prostate-cancer-grade-assessment/'
TRAIN_PATH    = DATA_PATH + 'train_images/'
MASKS_PATH    = DATA_PATH + 'train_label_masks/'
TEST_PATH     = DATA_PATH + 'test_images/'
MODEL_PATH    = './'                                       # path to save model
ENET_MODEL_PATH = '/kaggle/input/efficientnettf/'          # pretrained model from efficientnet package
MODEL_VERSION = 'v1.1'                                       # version for the model to be saved
restart       = True
restart_path  = '/kaggle/input/panda-modelweights/model_v1.0.h5'
EPOCHS        = 40

TILE_SIZE     = 128      # 16 128x128 tiles are selected from each image and mask
NUM_TILE      = 16
BATCH_SIZE    = 4
INI_LR        = 1.e-4

SEED          = 2020

Update to TensorFlow 2.2.0 and TensorFlow-Addons 0.10.0.
- CohenKappa metric and WeightedKappaLoose are used as implemented in TensorFlow Addons 0.10.0.

In [ ]:
import sys
print('Python {}'.format(sys.version))
print(sys.version_info)

In [ ]:
%%time
!pip uninstall tensorflow -y
!pip install --quiet /kaggle/input/tensorflow-addons/tensorboard_plugin_wit-1.6.0.post3-py3-none-any.whl
!pip install --quiet /kaggle/input/tensorflow-addons/tensorboard-2.2.2-py3-none-any.whl
!pip install --quiet /kaggle/input/tensorflow-addons/astunparse-1.6.3-py2.py3-none-any.whl
!pip install --quiet /kaggle/input/tensorflow-addons/gast-0.3.3-py2.py3-none-any.whl
!pip install --quiet /kaggle/input/tensorflow-addons/tensorflow_estimator-2.2.0-py2.py3-none-any.whl
!pip install --quiet /kaggle/input/tensorflow-addons/tensorflow-2.2.0-cp37-cp37m-manylinux2010_x86_64.whl

!pip install --quiet /kaggle/input/tensorflow-addons/typeguard-2.9.1-py3-none-any.whl
!pip install --quiet /kaggle/input/tensorflow-addons/tensorflow_addons-0.10.0-cp37-cp37m-manylinux2010_x86_64.whl

!pip install --quiet /kaggle/input/efficientnetrepo110/efficientnet-1.1.0-py3-none-any.whl

Check TensorFlow and TensorFlow-Addons version.

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
print('TensorFlow version:        {}'.format(tf.__version__))
print('TensorFlow-Addons version: {}'.format(tfa.__version__))

Load essential modules. 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow.keras as K
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
import albumentations as albu      # a fast image augmentation library

import skimage.io
import json

from tensorflow.keras import Model, Sequential
import efficientnet.tfkeras as efn

import os
import memory_profiler


Check GPUs availability. 

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    logical_gpus = tf.config.list_logical_devices('GPU')
    print('{} Physical GPUs, {} Logical GPUs'.format(len(gpus), len(logical_gpus)))

In [ ]:
os.listdir(DATA_PATH)

## Tiles of image

Load whole slide image(WSI), and select 16 128x128 tiles from each image according to the number of tissure pixels.

In [ ]:
def tile(img_id, mode = 'train', mask = False, tile_size = 128, num_tile = 16, aug = None):
    # This function selects <num_tile> tiles of size <tile_size>x<tile_size> 
    # for image of <img_id> (and mask) 
    # based on the maximum number of tissue pixels.
    if(mode == 'train'):
        img = skimage.io.MultiImage(os.path.join(TRAIN_PATH, img_id + '.tiff'))[-1] 
    elif(mode == 'test'):
        img = skimage.io.MultiImage(os.path.join(TEST_PATH, img_id + '.tiff'))[-1]
    else:
        raise AttributeError('tile mode Error')
    if aug:
        img = aug(image=img)['image']
    shape = img.shape
    pad0, pad1 = (tile_size - shape[0]%tile_size)%tile_size, (tile_size - shape[1]%tile_size)%tile_size
    img = np.pad(img, [[pad0//2, pad0-pad0//2], [pad1//2, pad1-pad1//2], [0,0]],
                 constant_values = 255)
    img = img.reshape(img.shape[0]//tile_size, tile_size, img.shape[1]//tile_size, tile_size,3)
    img = img.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size, 3)
    if len(img) < num_tile:
        img = np.pad(img, [[0, num_tile - len(img)], [0, 0], [0, 0], [0, 0]], 
                     constant_values = 255)    
    idxs = np.argsort(img.reshape(img.shape[0], -1).sum(-1))[:num_tile]
    img = np.array(img[idxs])/255.
    
    if(mask):
        mask = skimage.io.MultiImage(os.path.join(MASKS_PATH, img_id + '_mask.tiff'))[-1]
        if aug: 
            mask = aug(image=mask)['image']
        mask = np.pad(mask, [[pad0//2, pad0-pad0//2], [pad1//2, pad1-pad1//2], [0,0]],
                      constant_values = 0)
        mask = mask.reshape(mask.shape[0]//tile_size, tile_size, mask.shape[1]//tile_size, tile_size,3)
        mask = mask.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size, 3)
        if len(mask) < num_tile:
            mask = np.pad(mask, [[0, num_tile - len(img)], [0, 0], [0, 0], [0, 0]], 
                          constant_values = 0)
        mask = np.array(mask[idxs])
        return img, mask
    else:
        return img
    
def glue_to_one(tile_seq):
    l_tile = int(math.sqrt(NUM_TILE))
    img_glue = np.zeros((l_tile*TILE_SIZE, l_tile*TILE_SIZE, 3),
                         dtype = np.float32)
    for i, t in enumerate(tile_seq):
        x = i//l_tile
        y = i%l_tile
        img_glue[x*TILE_SIZE:(x+1)*TILE_SIZE,
                 y*TILE_SIZE:(y+1)*TILE_SIZE, :] = t
    return img_glue

In [ ]:
class PANDA_Sequence(Sequence):
    def __init__(self, df, batch_size=16, mode='fit', shuffle = False, aug = None, 
                 num_tile = 16, tile_size = 128, n_classes=6):
        self.df = df            # data frame with the image_id
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle
        self.aug = aug
        self.tile_size = tile_size
        self.num_tile = num_tile
        self.n_classes = n_classes
        self.l_tile = int(math.sqrt(self.num_tile))
        if(self.mode == 'fit'):
            self.tile_mode = 'train'
            self.tile_mask = False
        elif(self.mode == 'validate'):
            self.tile_mode = 'train'
            self.tile_mask = False
        elif(self.mode == 'predict'):
            self.tile_mode = 'test'
            self.tile_mask = False
        else:
            raise AttributeError('Sequence mode Error')
        self.on_epoch_end()
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    def __getitem__(self, index):
        X = np.zeros((self.batch_size, self.l_tile*self.tile_size, self.l_tile*self.tile_size, 3), dtype = np.float32)
        img_batch = self.df[index*self.batch_size : (index+1)*self.batch_size]['image_id'].values
        for i, img_id in enumerate(img_batch):
            img_tiles = tile(img_id, mode = self.tile_mode, mask = self.tile_mask, tile_size = self.tile_size,
                             num_tile = self.num_tile, aug = self.aug)
            X[i,] = glue_to_one(img_tiles)
        if self.mode in ['fit', 'validate']:
            y = np.zeros((self.batch_size, self.n_classes), dtype = np.float32)
            # encode label list
            lbls_batch = self.df[index * self.batch_size: (index+1) * self.batch_size]['isup_grade'].values
            for i in range(self.batch_size):
                y[i, lbls_batch[i]] = 1
            return X, y
        elif self.mode == 'predict':
            return X
        else:
            raise AttributeError('mode parameter error')        

## Load data

In [ ]:
print('Memory usage : {} MB'.format(*memory_profiler.memory_usage(-1)))

In [ ]:
df_train = pd.read_csv('{}/train.csv'.format(DATA_PATH))
print('train: {}'.format(df_train.shape))
print('unique isup grade: {}'.format(df_train['isup_grade'].nunique()))
X_train, X_val = train_test_split(df_train, test_size = .2, stratify=df_train['isup_grade'],
                                  random_state = SEED)
lbl_value_counts = X_train['isup_grade'].value_counts()
class_weights = {i: max(lbl_value_counts)/v for i, v in lbl_value_counts.items()}
print('classes weights: {}'.format(class_weights))
print('Memory usage : {} MB'.format(*memory_profiler.memory_usage(-1)))

Augmentation

In [ ]:
aug = albu.Compose([
    albu.OneOf([albu.RandomBrightness(limit=.15),
                albu.RandomContrast(limit=.3),
                albu.RandomGamma()], p=.25),
    albu.HorizontalFlip(p=.25),
    albu.VerticalFlip(p=.25),
    albu.ShiftScaleRotate(shift_limit = .1,
                          scale_limit=.1,
                          rotate_limit = 20,
                          p=.25)
])

In [ ]:
def plot_tiles(img_id):
    img = skimage.io.MultiImage(os.path.join(TRAIN_PATH, img_id + '.tiff'))[-1]
    img_tiles, mask_tiles = tile(img_id, mode = 'train', mask = True, tile_size = TILE_SIZE,
                                 num_tile = NUM_TILE)
    fig, ax = plt.subplots(1, 3, figsize=(18,12))
    fig.suptitle('Image ID: {}  data_provider: {}  ISUP grade: {}'.format(img_id, 
                                 df_train[df_train['image_id']==img_id]['data_provider'].values[0],
                                 df_train[df_train['image_id']==img_id]['isup_grade'].values[0]))
    ax[0].imshow(img)
    ax[1].imshow(glue_to_one(img_tiles))
    ax[2].imshow(glue_to_one(mask_tiles)[:,:,0], cmap = 'hot', vmin = 0, vmax = 5)
    for i in range(3):
        ax[i].axis('off')
    plt.show()
    return
    
#img_id = df_train[df_train['isup_grade']==3]['image_id'].sample(n=1, random_state = SEED).to_numpy()[0]
#plot_tiles(img_id)

Get statistics of images composed of tiles.

In [ ]:
def get_statistics(df, mode = 'train', tile_size = 128, num_tile = 16, aug = None):
    tile_avgs = []
    tile_sqavgs = []
    for img_id in df['image_id'].to_numpy():
        img_tiles = tile(img_id, mode = mode, mask = False, tile_size = tile_size, num_tile = num_tile, aug = aug)
        tile_avgs.append(img_tiles.reshape(-1,3).mean(axis=0))
        tile_sqavgs.append((img_tiles**2).reshape(-1,3).mean(axis=0))
    return np.array(tile_avgs), np.array(tile_sqavgs)

#tile_avgs, tile_sqavgs = get_statistics(df_train, mode = 'train', tile_size = TILE_SIZE, num_tile = NUM_TILE, aug = aug)
#tile_avg = tile_avgs.mean(axis = 0)
#tile_std = np.sqrt(tile_sqavgs.mean(axis = 0) - tile_avg**2)
tile_avg = [0.90413509, 0.81429153, 0.87288452]
tile_std = [0.13412014, 0.24779248, 0.16435623]
print('Tiles average {}'.format(tile_avg))
print('Tiles std.    {}'.format(tile_std))
     

In [ ]:
train_gen = PANDA_Sequence(
    df = X_train,
    batch_size = BATCH_SIZE,
    mode = 'fit',
    shuffle = False,
    aug = None,
    num_tile = NUM_TILE,
    tile_size = TILE_SIZE,
    n_classes = 6)

val_gen = PANDA_Sequence(
    df = X_val,
    batch_size = BATCH_SIZE,
    mode = 'validate',
    shuffle = False,
    aug = None,
    num_tile = NUM_TILE,
    tile_size = TILE_SIZE,
    n_classes = 6)


The generated data in a batch:

In [ ]:
Xt, yt = train_gen.__getitem__(0)
print('test X: {}'.format(Xt.shape))
print('test y: {}'.format(yt.shape))
fig, axes = plt.subplots(ncols = BATCH_SIZE, figsize=(18, 18))
for j in range(BATCH_SIZE):
    axes[j].imshow(Xt[j])
    axes[j].axis('off')
    axes[j].set_title('ISUP grade {}'.format(np.argmax(yt[j,])))
plt.show()
print('Memory usage : {} MB'.format(*memory_profiler.memory_usage(-1)))

## Model

Model based on EfficientNetB3:

In [ ]:
def get_uncompiled_model():
    conv_base = efn.EfficientNetB3(
        input_shape = (int(math.sqrt(NUM_TILE))*TILE_SIZE, int(math.sqrt(NUM_TILE))*TILE_SIZE, 3),
        weights = os.path.join(ENET_MODEL_PATH, 
                               'efficientnet-b3_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'),  # pretrained weights with ImageNet
        include_top = False,
        pooling = 'avg')
    conv_base = K.Model(inputs=conv_base.inputs, outputs=conv_base.outputs)
    model = Sequential()
    model.add(conv_base)
    #model.add(K.layers.Dropout(.2))
    model.add(K.layers.Dense(6, activation='softmax'))
    conv_base.trainable = False
    
    if restart:
        model.load_weights(restart_path, by_name = True, skip_mismatch = True)
        print('model weights loaded')
        # set more layers trainable
        conv_base.trainable = True
        set_trainable = False
        for layer in conv_base.layers:
            if layer.name == 'block7b_add':
                set_trainable = True
            if set_trainable:
                layer.trainable = True
            else:
                layer.trainable = False
    else:
        print('train from scratch')

    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer = K.optimizers.Adam(lr=INI_LR),
        loss = tfa.losses.WeightedKappaLoss(num_classes=6, weightage='quadratic'),
        metrics = ['categorical_accuracy', tfa.metrics.CohenKappa(num_classes=6, weightage='quadratic')]
        )
    return model

In [ ]:
model = get_compiled_model()
print('Memory usage : {} MB'.format(*memory_profiler.memory_usage(-1)))

In [ ]:
model.summary()

In [ ]:
K.utils.plot_model(model, show_shapes=True)

Plot of the top layers from EfficientNetB3 base model. Layer 'block7b_add' and above layers are trained. Lower layers are fixed at 'ImageNet' weights.  

In [ ]:
conv_base = efn.EfficientNetB3(
        input_shape = (int(math.sqrt(NUM_TILE))*TILE_SIZE, int(math.sqrt(NUM_TILE))*TILE_SIZE, 3),
        weights = os.path.join(ENET_MODEL_PATH, 
                               'efficientnet-b3_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'),  # pretrained weights with ImageNet
        include_top = False,
        pooling = 'avg')

_ = K.utils.plot_model(conv_base, to_file = 'model_ENetB3_notop.png', show_shapes=True)
im = plt.imread('model_ENetB3_notop.png')
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot()
ax.imshow(im)
ax.set_xlim(200,850)
ax.set_ylim(32800,32180)
ax.axis('off')
plt.show()

## Train the model

In [ ]:
%%time
model_file = '{}/model_{}.h5'.format(MODEL_PATH, MODEL_VERSION)
earlystopper = K.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    verbose = 1,
    mode = 'min'
)
modelsaver = K.callbacks.ModelCheckpoint(
    model_file,
    monitor = 'val_loss',
    verbose = 1,
    save_weights_only = True,
    save_best_only = True,
    mode = 'min'
)
lrreducer = K.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = .1,
    patience = 5,
    verbose = 1,
    min_lr = 1.e-7
)

class mem_use(K.callbacks.Callback):    
    def on_epoch_end(self, epoch, logs=None):
        mem_usage = memory_profiler.memory_usage(-1)
        format_str = 'Memory usage at epoch {:8d}: ' + '{:8.0f}'*len(mem_usage) + ' MB'
        print(format_str.format(epoch, *mem_usage))
        


history = model.fit(
    train_gen,
    validation_data = val_gen,
    class_weight = class_weights,
    callbacks = [earlystopper, modelsaver, lrreducer, mem_use()],
    epochs = EPOCHS,
    verbose = 1
    )


In [ ]:
history_file = 'history_{}.txt'.format(MODEL_VERSION)
dict_to_save = {}
for k, v in history.history.items():
    dict_to_save.update({
        k: [np.format_float_positional(x) for x in history.history[k]]
    })
with open(history_file, 'w') as file:
    json.dump(dict_to_save, file)
ep_max = EPOCHS
plt.plot(history.history['loss'][:ep_max], label='loss')
plt.plot(history.history['val_loss'][:ep_max], label='val_loss')
plt.legend()
plt.show()
plt.plot(history.history['categorical_accuracy'][:ep_max], label='accuracy')
plt.plot(history.history['val_categorical_accuracy'][:ep_max], label='val_accuracy')
plt.plot(history.history['cohen_kappa'][:ep_max], label = 'cohen_kappa')
plt.plot(history.history['val_cohen_kappa'][:ep_max], label = 'val_cohen_kappa ')
plt.legend()
plt.show()

In [ ]:
mem_usage = memory_profiler.memory_usage(-1)
format_str = 'Memory usage: ' + '{:8.0f}'*len(mem_usage) + ' MB'
print(format_str.format(*mem_usage))
        

## Inference

In [ ]:
df_test = pd.read_csv('{}/test.csv'.format(DATA_PATH))
print(df_test.shape)
pred = np.zeros((len(df_test), 6))
if os.path.exists(TEST_PATH):
    sub_gen = PANDA_Sequence(
        df = df_test,
        batch_size = 1,
        mode = 'predict',
        shuffle = False,
        aug = None,
        num_tile = NUM_TILE,
        tile_size = TILE_SIZE,
        n_classes = 6
    )
    pred = model.predict(sub_gen)
    print('Predict for {} images'.format(len(pred)))
else:
    print('Predict zeros')

df_test['isup_grade'] = np.argmax(pred, axis = 1)
df_test.drop('data_provider', axis = 1, inplace = True)
df_test.to_csv('submission.csv', index = False)
print('submission saved')

In [ ]:
!head /kaggle/working/submission.csv